<a href="https://colab.research.google.com/github/dk-wei/recommendation-algo-implementation/blob/main/FFM_w_xlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在推荐系统、搜索排序、效果广告等场景中，点击率预估是十分重要的部分，CTR 算法也被誉为镶嵌在互联网技术上的明珠。在深度学习火热之前，除了简单的 LR 以外常用的算法类有：以决策树为主的 Boosting 算法；以因子分解为基础的 FM 算法。相对而言，树模型比较适合学习数值类的连续特征，而后者更适合学习 ID 类（Categorical ）的稀疏特征。深度学习横空出世之后，更多的研究者把重心放在如何将 Deep Learning 运用于 CTR 任务，比较著名的有 Google 开源的 Wide and Deep 算法和 DCN 网络。通常来说，FM-based 算法更适合深度学习，深度学习需要海量的训练数据，大量稀疏的 ID 类特征为深度学习提供绝佳的土壤。

FM 的主要目的是解决稀疏特征下参数学习问题，可以实现树模型不能学习的特征交叉。本文主要涉及几个以 FM 为基础的算法。本着学习交流分享精神，本文从 FM 出发，梳理一下这些以 FM 为基础的算法。如有纰漏，敬请指出。

使用CRITEO_CTR比赛的简略数据，构建简单的FFM模型，

In [1]:
pip install xlearn

In [2]:
import xlearn as xl
import pandas as pd

In [1]:
!git clone https://github.com/wngaw/blog.git

fatal: destination path 'blog' already exists and is not an empty directory.


In [5]:
cd /content/blog/xlearn_example/src

/content/blog/xlearn_example/src


In [6]:
import pandas as pd
import xlearn as xl
from sklearn.model_selection import train_test_split
from utils import _convert_to_ffm
import config
import warnings
warnings.filterwarnings('ignore')

In [7]:
train = pd.read_csv('data/train.csv')

In [8]:
cols = ['Education','ApplicantIncome','Loan_Status','Credit_History']
train_sub = train[cols]
train_sub['Credit_History'].fillna(0, inplace = True)
dict_ls = {'Y':1, 'N':0}
train_sub['Loan_Status'].replace(dict_ls, inplace = True)

In [9]:
train, test = train_test_split(train_sub, test_size = 0.3, random_state = 5)

In [10]:
print(f' train data: \n {train.head()}')

 train data: 
         Education  ApplicantIncome  Loan_Status  Credit_History
16   Not Graduate             3596            1             0.0
461      Graduate             7740            1             1.0
285      Graduate             3158            1             1.0
156      Graduate             6000            1             0.0
384      Graduate             4566            0             1.0


In [11]:
encoder = {"currentcode": len(config.NUMERICAL_FEATURES),
           "catdict": {},
           "catcodes": {}}

In [12]:
encoder = _convert_to_ffm('data/', train, 'train', config.GOAL[0],
                          config.NUMERICAL_FEATURES,
                          config.CATEGORICAL_FEATURES,
                          config.ALL_FEATURES,
                          encoder)

encoder = _convert_to_ffm('data/', test, 'test', config.GOAL[0],
                          config.NUMERICAL_FEATURES,
                          config.CATEGORICAL_FEATURES,
                          config.ALL_FEATURES,
                          encoder)

convert_to_ffm - START
UPDATING CATDICT: numeric field - ApplicantIncome
UPDATING CATDICT: categorical field - Education
UPDATING CATDICT: categorical field - Credit_History
UPDATING CATCODES: categorical field - Education
UPDATING CATCODES: categorical value for field Education - Not Graduate
UPDATING CATCODES: categorical field - Credit_History
UPDATING CATCODES: categorical value for field Credit_History - 0.0
UPDATING CATCODES: categorical value for field Education - Graduate
UPDATING CATCODES: categorical value for field Credit_History - 1.0
convert_to_ffm - START


In [ ]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("data/train_ffm.txt")
ffm_model.setValidate("data/test_ffm.txt")
param = {'task': 'binary',
         'lr': 0.2,
         'lambda': 0.002,
         'metric': 'auc'}
ffm_model.fit(param, 'trained_models/model.out')

In [14]:
param = {'task': 'binary',
         'lr': 0.2,
         'lambda': 0.002,
         'metric': 'auc'}
         
ffm_model = xl.create_ffm()
#ffm_model.cv(param)

In [ ]:
ffm_model.setTest("data/test_ffm.txt")
ffm_model.setSigmoid()
ffm_model.predict("trained_models/model.out", "output/predictions.txt")